# Энигма

## Parts of Enigma

In [1]:
class Alphabet:
    def __init__(self, alphabet: str) -> None:
        self._alphabet = alphabet

    def get_all(self) -> str:
        return self._alphabet

    def char2idx(self, char: str) -> int:
        return self._alphabet.find(char)
    
    def idx2char(self, idx: int) -> str:
        return self._alphabet[idx]
    
    def __len__(self) -> int:
        return len(self._alphabet)

In [2]:
class Rotor:
    def __init__(self, alphabet: Alphabet, shift: int) -> None:
        self._initial_shift = shift
        self._shift = shift
        self._alphabet = alphabet

    def __call__(self, char: str, reverse: bool = False) -> str:
        pos = self._alphabet.char2idx(char)
        new_pos = (pos - self._shift if reverse else pos + self._shift) % len(self._alphabet)
        return self._alphabet.idx2char(new_pos)

    def increment(self) -> bool:
        self._shift += 1
        if self._shift >= len(self._alphabet):
            self._shift = 0
            return True
        return False

    def reset(self) -> None:
        self._shift = self._initial_shift

In [3]:
class Reflector:
    def __init__(self, alphabet: Alphabet, mapping: dict[str, str]) -> None:
        self._alphabet = alphabet
        self._mapping = mapping
    
    def __call__(self, char: str) -> str:
        return self._mapping[char]

In [4]:
class Plugboard:
    def __init__(self, repl_table: dict[str, str]) -> None:
        self._repl_table = repl_table
    
    def __call__(self, char: str) -> str:
        return self._repl_table[char]

In [5]:
class Enigma:
    def __init__(self, plugboard: Plugboard, rotors: list[Rotor], reflector: Reflector) -> None:
        self._plugboard = plugboard
        self._rotors = rotors
        self._reflector = reflector
    
    def reset_rotors(self):
        for rotor in self._rotors:
            rotor.reset()
    
    def encode(self, text: str) -> str:
        ciphertext = []
        for c in text:
            # passing through the plugboard 
            cipher_c = self._plugboard(c)

            # passing through rotors (forward)
            for r in self._rotors:
                cipher_c = r(cipher_c)
        
            # passing through the reflector
            cipher_c = self._reflector(cipher_c)

            # passing back through the rotors (reverse)
            for r in reversed(self._rotors):
                cipher_c = r(cipher_c, reverse=True)
            
            # passing through the plugboard after reflection
            cipher_c = self._plugboard(cipher_c)
            
            ciphertext.append(cipher_c)
            # moving rotors 1 step forward
            for r in self._rotors:
                if not r.increment():
                    break

        return "".join(ciphertext)

## Setting Up

In [6]:
import string

alphabet = Alphabet(string.ascii_lowercase + string.ascii_uppercase + " ")

rotors = [Rotor(alphabet, i) for i in range(1, 4)]

repl_table = {c: c for c in alphabet.get_all()}
replacements = [('a', 'b'), ('c', 'd'), ('e', 'f')]
for (c1, c2) in replacements:
    repl_table[c1] = c2
    repl_table[c2] = c1

plugboard = Plugboard(repl_table)

reflector_mapping = {char: char for char in alphabet.get_all()}
reflector_mapping.update({
    alphabet.idx2char(i): alphabet.idx2char(len(alphabet) - i - 1) for i in range(len(alphabet))
})

reflector = Reflector(alphabet, reflector_mapping)

enigma = Enigma(plugboard, rotors, reflector)

## Encoding/Decoding

In [7]:
enigma.reset_rotors()

encoded_message = enigma.encode("Hello")
print(f"Encoded: {encoded_message}")

enigma.reset_rotors()

decoded_message = enigma.encode(encoded_message)
print(f"Decoded: {decoded_message}")

Encoded: hHzxs
Decoded: Hello
